# Installing modules and packages

In [8]:
import tkinter as tk
from tkinter import messagebox, filedialog
from PIL import Image, ImageTk
import cv2
from ultralytics import YOLO

# Loading YOLOv8 model

In [9]:
model = YOLO("yolov8n.pt")

# GUI window setup

In [10]:
root = tk.Tk()
root.title("Single Frame Object Detection")
root.geometry("700x650")
root.resizable(False, False)

''

# GUI elements

In [11]:
image_label = tk.Label(root)
image_label.pack(pady=10)

detected_label = tk.Label(root, text="", font=("Helvetica", 12), fg="blue", wraplength=750, justify="left")
detected_label.pack(pady=5)

# Saving function

In [12]:
last_detected_image = None
# Detect function
def detect_single_frame():
    global last_detected_image

    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 768)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 432)
    ret, frame = cap.read()
    cap.release()

    if not ret:
        messagebox.showerror("Error", "Failed to capture image from webcam.")
        return

    results = model.predict(source=frame, conf=0.25)

    for r in results:
        frame = r.plot()
        names = r.names
        class_ids = r.boxes.cls.tolist() if r.boxes else []
        detected_classes = [names[int(cls_id)] for cls_id in class_ids]
        detected_text = ", ".join(sorted(set(detected_classes)))
        detected_label.config(text=f"Detected: {detected_text}" if detected_classes else "No objects detected.")

     
    rgb_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(rgb_image)
    tk_image = ImageTk.PhotoImage(pil_image)

    last_detected_image = pil_image

    image_label.imgtk = tk_image
    image_label.configure(image=tk_image)


In [13]:
def save_image():
    global last_detected_image
    if last_detected_image is None:
        messagebox.showwarning("No Detection", "Run a detection before saving.")
        return

    file_path = filedialog.asksaveasfilename(defaultextension=".jpg", filetypes=[("JPEG Image", "*.jpg"), ("PNG Image", "*.png")])
    if file_path:
        last_detected_image.save(file_path)
        messagebox.showinfo("Saved", f"Image saved successfully:\n{file_path}")


def clear_output():
    global last_detected_image
    image_label.config(image='')
    image_label.imgtk = None
    detected_label.config(text='')
    last_detected_image = None


# Buttons & Starting GUI

In [ ]:
button_frame = tk.Frame(root)
button_frame.pack(pady=15)

detect_btn = tk.Button(button_frame, text="Start Detection", command=detect_single_frame, bg="green", fg="white", width=20)
detect_btn.grid(row=0, column=0, padx=10)

save_btn = tk.Button(button_frame, text="Save Image", command=save_image, bg="blue", fg="white", width=12)
save_btn.grid(row=0, column=1, padx=10)

clear_btn = tk.Button(button_frame, text="Clear", command=clear_output, bg="gray", fg="white", width=10)
clear_btn.grid(row=0, column=2, padx=10)

exit_btn = tk.Button(button_frame, text="Exit", command=root.destroy, bg="red", fg="white", width=10)
exit_btn.grid(row=0, column=3, padx=10)


root.mainloop()


0: 480x640 2 persons, 1 tv, 85.0ms
Speed: 4.9ms preprocess, 85.0ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)
